r178459r MUJATI KUNDAI B HCT404 CTHSC

In [4]:
import numpy as np
import pandas as pd

In [5]:
from google.colab import files
uploaded = files.upload()

Saving spam_or_not_spam.csv to spam_or_not_spam.csv


In [6]:
import io
df = pd.read_csv(io.BytesIO(uploaded['spam_or_not_spam.csv']))
# Dataset is now stored in a Pandas Dataframe

In [7]:
df.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [9]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)
print(len(train_set), "train +", len(test_set), "test")

2400 train + 600 test


In [10]:
from nltk.corpus import stopwords
import string
import re
def load_doc(filename):
# open the file as read only
  file = open(filename, 'r' )
# read all text
  text = file.read()
# close the file
  file.close()
  return text

In [11]:
def clean_doc(doc):
# split into tokens by white space
  tokens = doc.split()
# prepare regex for char filtering
  re_punc = re.compile( ' [%s] ' % re.escape(string.punctuation))
# remove punctuation from each word
  tokens = [re_punc.sub( '' , w) for w in tokens]
# remove remaining tokens that are not alphabetic
  tokens = [word for word in tokens if word.isalpha()]
# filter out stop words
  stop_words = set(stopwords.words( 'english' ))
  tokens = [w for w in tokens if not w in stop_words]
# filter out short tokens
  tokens = [word for word in tokens if len(word) > 1]
  return tokens

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
filename = a
text = load_doc(filename)
tokens = clean_doc(text)
print(tokens)

TypeError: ignored

In [16]:
df1 = df['email']
df1

0        date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...
1       martin a posted tassos papadopoulos the greek ...
2       man threatens explosion in moscow thursday aug...
3       klez the virus that won t die already the most...
4        in adding cream to spaghetti carbonara which ...
                              ...                        
2995     abc s good morning america ranks it the NUMBE...
2996     hyperlink hyperlink hyperlink let mortgage le...
2997     thank you for shopping with us gifts for all ...
2998     the famous ebay marketing e course learn to s...
2999     hello this is chinese traditional 子 件 NUMBER世...
Name: email, Length: 3000, dtype: object

In [20]:
a = list(df['email'])

In [21]:
a

[' date wed NUMBER aug NUMBER NUMBER NUMBER NUMBER NUMBER from chris garrigues cwg dated NUMBER NUMBERfaNUMBERd deepeddy com message id NUMBER NUMBER tmda deepeddy vircio com i can t reproduce this error for me it is very repeatable like every time without fail this is the debug log of the pick happening NUMBER NUMBER NUMBER pick_it exec pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER NUMBER NUMBER exec pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER NUMBER NUMBER ftoc_pickmsgs NUMBER hit NUMBER NUMBER NUMBER marking NUMBER hits NUMBER NUMBER NUMBER tkerror syntax error in expression int note if i run the pick command by hand delta pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER hit that s where the NUMBER hit comes from obviously the version of nmh i m using is delta pick version pick nmh NUMBER NUMBER NUMBER compiled on URL at sun mar NUMBER NUMBER

In [ ]:
# load document
in_filename = '/content/drive/MyDrive/Colab Notebooks/datasets/pg1497.txt'
doc = load_doc(in_filename)
print(doc[:200])

In [ ]:
# turn a doc into clean tokens
def clean_doc(doc):
  # replace ' -- ' with a space ' '
  doc = doc.replace( '--' , ' ' )

  # split into tokens by white space
  tokens = doc.split()

  # prepare regex for char filtering
  re_punc = re.compile( '[%s]' % re.escape(string.punctuation))

  # remove punctuation from each word
  tokens = [re_punc.sub( '' , w) for w in tokens]

  # remove remaining tokens that are not alphabetic
  tokens = [word for word in tokens if word.isalpha()]

  # make lower case
  tokens = [word.lower() for word in tokens]
  return tokens

In [ ]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print(f"Len Tokens: {len(tokens)}")
print(f'Total Tokens: {tokens}')
print(f'Len Unique Tokens: {len(set(tokens))}')
print(f'Unique Tokens: {set(tokens)}')

In [ ]:
filename = a
# save tokens to file, one dialog per line
def save_doc(lines, filename):
  data = '\n' .join(lines)
  file = open(filename, 'w' )
  file.write(data)
  file.close()

In [ ]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
  # select sequence of tokens
  seq = tokens[i-length:i]

  # convert into a line
  line = ' '.join(seq)

  # store
  sequences.append(line)

print(f'Total Sequences: {len(sequences)}')
# save sequences to file
out_filename = '/content/drive/MyDrive/Colab Notebooks/LM/republic_sequences.txt'
save_doc(sequences, out_filename)

# load
in_filename = '/content/drive/MyDrive/Colab Notebooks/LM/republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [ ]:
# load
in_filename = '/content/drive/MyDrive/Colab Notebooks/LMrepublic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split( '\n' )
lines[:5]

In [ ]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [ ]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
print(sequences[:1])

In [ ]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [ ]:
def define_model(vocab_size, seq_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 50, input_length=seq_length))
  model.add(LSTM(100, return_sequences=True))
  model.add(LSTM(100))
  model.add(Dense(100, activation= 'relu' ))
  model.add(Dense(vocab_size, activation= 'softmax'))
  model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
  model.summary()
  plot_model(model, to_file= '/content/drive/MyDrive/Colab Notebooks/LM/model.png' , show_shapes=True)
  return model

In [ ]:
model = define_model(vocab_size, seq_length)

In [ ]:
model.fit(X, y, batch_size=128, epochs=100)

In [ ]:
# save the model to file
model.save('/content/drive/MyDrive/Colab Notebooks/LM/model.h5')
# save the tokenizer
with open('/content/drive/MyDrive/Colab Notebooks/LM/tokenizer.pkl' , 'wb') as token_sv:
  dump(tokenizer, token_sv) 

In [ ]:
_, acc = model.evaluate(X, y, verbose=0)
print(f'Train Accuracy: {(acc*100)}')

In [ ]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# load the model
model = load_model( '/content/drive/MyDrive/Colab Notebooks/LM/model.h5')
# load the tokenizer
tokenizer = load(open('/content/drive/MyDrive/Colab Notebooks/LM/tokenizer.pkl', 'rb'))
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n' )

In [ ]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
  result = list()
  in_text = seed_text

  # generate a fixed number of words
  for _ in range(n_words):
    # encode the text as integer
    encoded = tokenizer.texts_to_sequences([in_text])[0]

    # truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating= 'pre' )

    # predict probabilities for each word
    yhat = model.predict_classes(encoded, verbose=0)

    # map predicted word index to word
    out_word = ''
    for word, index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
    # append to input
    in_text += ' ' + out_word
    result.append(out_word)
  return ' ' .join(result)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n' )

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 10)
print(generated)